**Importing Libraries**

In [ ]:
import os
import time
import random
import math
import tqdm
from tqdm import tqdm
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import PIL
from PIL import Image,ImageEnhance,ImageOps,ImageFilter
import tensorflow as tf
import keras
from keras import backend as K
from keras.optimizers import Adam
from keras.models import Model,Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.applications import MobileNet
from keras.optimizers import Adam

**Defining Paths**

In [ ]:
root_path=os.getcwd()
train_path=root_path+r"/tomato/train/"
validation_path=root_path+r"/tomato/val/"

**Printing Datset Summary**

In [ ]:
def print_dataset_summary(dataset_path):

    print("-"*100,"\n")

    count = 0

    class_count = {}

    for dirs in tqdm(os.listdir(dataset_path)):
        for files in os.listdir(dataset_path + dirs):
            class_count.setdefault(dirs, 0)
            class_count[dirs] += 1
            count += 1

    print("\n\nTotal images :{}\n".format(count))

    total_length = 60

    for item in class_count:
        print(item + "-" * (total_length - len(item)) + str(class_count[item]))
    print()

print_dataset_summary(train_path)
print_dataset_summary(validation_path)

---------------------------------------------------------------------------------------------------- 



100%|██████████| 10/10 [00:00<00:00, 6404.50it/s]




Total images :219

Tomato___Early_blight---------------------------------------20
Tomato___Spider_mites Two-spotted_spider_mite---------------20
Tomato___Target_Spot----------------------------------------17
Tomato___Late_blight----------------------------------------19
Tomato___healthy--------------------------------------------20
Tomato___Bacterial_spot-------------------------------------19
Tomato___Tomato_Yellow_Leaf_Curl_Virus----------------------28
Tomato___Tomato_mosaic_virus--------------------------------24
Tomato___Leaf_Mold------------------------------------------27
Tomato___Septoria_leaf_spot---------------------------------25

---------------------------------------------------------------------------------------------------- 



100%|██████████| 10/10 [00:00<00:00, 20350.82it/s]



Total images :219

Tomato___Early_blight---------------------------------------20
Tomato___Spider_mites Two-spotted_spider_mite---------------20
Tomato___Target_Spot----------------------------------------17
Tomato___Late_blight----------------------------------------19
Tomato___healthy--------------------------------------------20
Tomato___Bacterial_spot-------------------------------------19
Tomato___Tomato_Yellow_Leaf_Curl_Virus----------------------28
Tomato___Tomato_mosaic_virus--------------------------------24
Tomato___Leaf_Mold------------------------------------------27
Tomato___Septoria_leaf_spot---------------------------------25



**Defining Classes**

In [ ]:
CLASSES=[
            'Tomato___Bacterial_spot',
            'Tomato___Early_blight',
            'Tomato___healthy',
            'Tomato___Late_blight',
            'Tomato___Leaf_Mold',
            'Tomato___Septoria_leaf_spot',
            'Tomato___Spider_mites_Two_spotted_spider_mite',
            'Tomato___Target_Spot',
            'Tomato___Tomato_mosaic_virus',
            'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
]

**Creating Train and Validation Files**

In [ ]:
def create_files(train_path,validation_path,class_list):

    with open("train.txt","w") as train_file:

        for dirs in tqdm(os.listdir(train_path)):
            for images in os.listdir(train_path+dirs):
                image_path=train_path+dirs+"/"+images+","+dirs
                train_file.writelines(image_path+"\n")

        print("\nSuccessfully created train.txt\n")
        time.sleep(0.1)


    with open("val.txt","w") as validation_file:

        for dirs in tqdm(os.listdir(validation_path)):
            for images in os.listdir(validation_path+dirs):
                image_path=validation_path+dirs+"/"+images+","+dirs
                validation_file.writelines(image_path+"\n")

        print("\nSuccessfully created val.txt")

create_files(train_path,validation_path,CLASSES)

100%|██████████| 11/11 [00:00<00:00, 1242.05it/s]



Successfully created train.txt



100%|██████████| 11/11 [00:00<00:00, 1764.06it/s]


Successfully created val.txt


**Testing Our Files**

In [ ]:
def test_images(file_path,total_images):

    with open(file_path,"r") as test_file:

            for lines in test_file.readlines()[:total_images]:
                curr_image_path=lines.replace("\n","").split(",")

                image=Image.open(curr_image_path[0])
                plt.title(curr_image_path[-1])
                plt.imshow(image)
                plt.show()

**Class for Image Augmentation**

In [ ]:
class Augmentation():

    def __init__(self):

        self.zoom_percentage=0.3

    def get_prob(self):
        return random.random()

    def enhance(self,image):

        self.factor = random.uniform(1,1.5)

        if (self.get_prob()>0.4):
            image=ImageEnhance.Brightness(image).enhance(self.factor)

        if (self.get_prob()>0.5):
            image=ImageEnhance.Contrast(image).enhance(self.factor)

        if (self.get_prob()>0.4):
            image=ImageEnhance.Sharpness(image).enhance(self.factor)

        return image

    def rotate(self,image):

        self.degree = random.choice([90,180,270])
        return image.rotate(self.degree)

    def flip(self,image):

        if (self.get_prob()>0.3):
            image = image.transpose(Image.FLIP_LEFT_RIGHT)

        if (self.get_prob()>0.4):
            image=image.transpose(Image.FLIP_TOP_BOTTOM)

        return image

    def enlarge(self,image):

        W,H=image.size

        orig_area=W*H

        width=math.ceil(math.sqrt(orig_area*(1-self.zoom_percentage)))
        height=width

        x1=max(math.ceil((W-width)/2),0)
        y1=max(math.ceil((W-width)/2),0)
        x2=min(math.ceil(width+(W-width)/2),W)
        y2=min(math.ceil(height+(W-width)/2),H)

        image=image.crop((x1,y1,x2,y2))

        return image

    def transform(self,image):

        if (self.get_prob()>0.3):
            image=self.rotate(image)

        if (self.get_prob()>0.4):
            image=self.flip(image)

        if (self.get_prob()>0.2):
            image=self.enhance(image)

        if (self.get_prob()>0.3):
            image=self.enlarge(image)

        return image


**DataGenerator Class**

In [ ]:
class DataGenerator(tf.keras.utils.Sequence,Augmentation):

    def __init__(self,labels,mode):
        self.mode=mode
        self.batch_size=4
        self.labels=labels
        self.all_images=[]
        self.all_classes=[]
        self.target_size=(224,224,3)

        super().__init__()

        self.CLASSES=[
            'Tomato___Bacterial_spot',
            'Tomato___Early_blight',
            'Tomato___healthy',
            'Tomato___Late_blight',
            'Tomato___Leaf_Mold',
            'Tomato___Septoria_leaf_spot',
            'Tomato___Spider_mites_Two_spotted_spider_mite',
            'Tomato___Target_Spot',
            'Tomato___Tomato_mosaic_virus',
            'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
]

        with open(labels,"r+") as f:
            contents=f.readlines()
            random.shuffle(contents)
            for lines in contents:
                lines=lines.replace("\n","").split(",")
                self.all_images.append(lines[0])
                # print(lines[-1])
                self.all_classes.append(self.CLASSES.index(lines[-1]))

    def generate_index(self,idx):
        return (self.all_images[idx*self.batch_size:self.batch_size*idx+self.batch_size] ,
                    self.all_classes[idx*self.batch_size:self.batch_size*idx+self.batch_size])

    def preprocess(self,image):
        # image=image.resize((self.target_size[0],self.target_size[1]))
        if (self.mode):
            image=self.transform(image)

        image=ImageEnhance.Sharpness(image).enhance(3) # Sharpening Each Image of train set

        image=image.resize((self.target_size[0],self.target_size[1]))
        image=np.array(image)
        image=image/image.max()
        return image

    def shuffle(self,images,labels):

        temp_list=list(zip(images,labels))
        random.shuffle(temp_list)
        self.all_images,self.all_classes=zip(*temp_list)



    def get_images_with_labels(self,idx):

        im_path,cls=self.generate_index(idx)

        self.images=[]
        self.cls=[]

        for fname,label in zip(im_path,cls):
            image=Image.open(fname).convert("RGB")
            image=self.preprocess(image)
            self.images.append(np.array(image))
            zeros=np.zeros(10)
            zeros[label]=1.0
            self.cls.append(zeros)

        return (
                np.asarray(self.images).reshape(self.batch_size,self.target_size[0],self.target_size[1],self.target_size[2]),
                np.asarray(self.cls).reshape(self.batch_size,-1)
                )


    def __getitem__(self,index):

        images,labels=self.get_images_with_labels(index)
        return (images,labels)

    def __len__(self):
        return len(self.all_images)//self.batch_size


    def on_epoch_end(self):
      self.shuffle(self.all_images,self.all_classes)



**Initializing Train Generator**

In [ ]:
train_generator=DataGenerator("train.txt",1)

**Testing Our train Generator**

In [ ]:
(images,labels)=train_generator.__getitem__(0)

for image,label in zip(images,labels):
    plt.imshow(image)
    plt.title(CLASSES[np.argmax(label)])
    plt.show()

**Initializing Validation Generator**

In [ ]:
val_generator=DataGenerator("val.txt",0)

In [ ]:
(images,labels)=val_generator.__getitem__(1)
print(images.shape)
print(labels.shape)
for image,label in zip(images,labels):
    plt.imshow(image)
    plt.title(CLASSES[np.argmax(label)])
    plt.show()

**Defining Model (MobileNet)**

In [ ]:
base_model=MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3))

# Freezing all the layers of the base model

for layers in base_model.layers:
    layers.trainable=False

17225924/17225924 [==============================] - 0s 0us/step


**Adding Dense Layers**

In [ ]:
# last_layer = base_model.output
x = base_model.output

x=Flatten()(x)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.2)(x)
preds=Dense(10,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

**Creating folder for saving model checkpoints**

In [ ]:
!mkdir "/checkpoint/"

**Defining Callbacks**

In [ ]:
checkpoint_filepath = '/checkpoint/model-epoch:{epoch:03d}-val_accuracy:{val_accuracy:.4f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callbacks=[model_checkpoint_callback]

**Fitting the model on the train data (Training starts here...)**

In [ ]:
history=model.fit(train_generator,
          validation_data=val_generator,
          steps_per_epoch=len(train_generator),
          validation_steps=len(val_generator),
          callbacks=callbacks,
          epochs=100)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
def test_on_images(image_path):

    image=Image.open(image_path).convert("RGB")
    print(image.mode)
    image=image.resize((224,224))
    image=ImageEnhance.Sharpness(image).enhance(2)
    # image= image.filter(ImageFilter.GaussianBlur(radius = 2))
    image=np.array(image).reshape(-1,224,224,3)
    image=image/image.max()

    prediction=model.predict(image)
    accuracy=round(np.max(prediction)*100,3)
    prediction=CLASSES[np.argmax(prediction)]
    plt.imshow(image.reshape(224,224,3))
    plt.title(prediction+":"+str(accuracy)+"%")
    plt.show()